In [1]:
using JuMP, Gurobi
using Statistics, StatsBase

In [2]:
# import network parameters
include("ieee_30_case.jl")

# Construct space-time network
# The jth spatial node at time i is = [(i-1)*N+j]th space-time node

Ast_tvl = zeros(N*T,N*T)        # adjacency matrix of space-time nodes
Fst_tvl = zeros(N*T,N*T)        # capacity matrix of temporal vritual links
r = ones(T,N) * 20              # reserved computing power

# Encode temporal virtual link information

# This loop only adds temporal virtual links between a node at t and
# itself at t+1 with a capacity of 10
# This applies to all spatial nodes
for i in 1:T-1
    for j in 1:N
        if D[j] > 0
            from_index = (i-1)*N+j
            to_index = i*N+j
            Ast_tvl[from_index,to_index] = 1
            Fst_tvl[from_index,to_index] = 10
        end
    end
end

α_γ = 3 * copy(Fst_tvl .> 0);
theta_diff_lim = 10;

In [14]:
# Optimization formulation

m_v = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(m_v, s[1:T,1:N] >= 0)                         # supply
@variable(m_v, d[1:T,1:N] >= 0)                         # demand
@variable(m_v, f[1:N,1:N,1:T] >= 0)                     # flow
@variable(m_v, del[1:N,1:N,1:T] >= 0)                   # spatial virtual shift
@variable(m_v, gam[1:N*T,1:N*T] >= 0)                   # temporal virtual shift
@variable(m_v, theta[1:T,1:N])

# Flow constraints and spatial virtual shift constraints
for t in 1:T
    for i in 1:N, j in 1:N
        @constraint(m_v, f[i,j,t] <= F[i,j])            # flow capacity constraints
        @constraint(m_v, del[i,j,t] <= Del[i,j])        # spatial virtual shift capacity constraints
        #@constraint(m_v, del[i,j,t] == 0)

        if i > j && B[i,j] > 0
            @constraint(m_v, f[i,j,t] - f[j,i,t] == B[i,j]*(theta[t,j] - theta[t,i]))
            @constraint(m_v, theta[t,j] - theta[t,i] <= theta_diff_lim)
            @constraint(m_v, theta[t,i] - theta[t,j] <= theta_diff_lim)
        end
    end
end

# Temporal virtual shift constraints
for i in 1:N*T, j in 1:N*T
    if Ast_tvl[i,j] == 0
        @constraint(m_v, gam[i,j] == 0)
    else
        @constraint(m_v, gam[i,j] <= Fst_tvl[i,j])       # temporal shift is one-directional
        #@constraint(m_v, gam[i,j] <= 0)       # temporal shift is one-directional
    end
end

# Supply and demand capacity constraints
for i in 1:T, j in 1:N
    @constraint(m_v, s[i,j] <= S_st[i,j])
    @constraint(m_v, d[i,j] <= D_st[i,j])
end

for i in 1:T, j in 1:N
    @constraint(m_v, d[i,j] - sum(del[j,:,i]) + sum(del[:,j,i]) + sum(gam[:,(i-1)*N+j]) - sum(gam[(i-1)*N+j,:]) <= D_st[i,j] + r[i,j])
    @constraint(m_v, d[i,j] - sum(del[j,:,i]) + sum(del[:,j,i]) + sum(gam[:,(i-1)*N+j]) - sum(gam[(i-1)*N+j,:]) >= 0)
end

# Node balance constraint
bals = Dict()
for i in 1:T
    for j in 1:N
        bals[(i,j)] = @constraint(m_v, s[i,j] - d[i,j] - sum(f[j,:,i]) + sum(f[:,j,i]) + sum(del[j,:,i]) - sum(del[:,j,i]) - sum(gam[:,(i-1)*N+j]) + sum(gam[(i-1)*N+j,:]) == 0)
    end
end

# Objecitve: social welfare
@objective(m_v, Min, sum(α_s_st.*s - α_d_st.*d) + sum(α_f.*f) + sum(α_δ.*del) + sum(α_γ.*gam));

Academic license - for non-commercial use only


In [15]:
println("=======With virtual shift capacity======")
optimize!(m_v)
println("Social welfare: ", -objective_value(m_v))
prices_vl = zeros(T,N)
for i in 1:T, j in 1:N
    prices_vl[i,j] = dual(bals[(i,j)])
end

for i in 1:T
    println("Nodal prices at time $(i): ", prices_vl[i,:])
end

=======With virtual shift capacity======
Academic license - for non-commercial use only
Social welfare: 11216.925696855025
Nodal prices at time 1: [17.4754, 20.3723, 28.8844, 28.8844, 24.4775, 23.5002, 28.8844, 28.8845, 24.6898, 28.8844, 29.6898, 29.7635, 31.0985, 28.8845, 29.6159, 30.8778, 31.5646, 30.1474, 28.8845, 28.8845, 32.8864, 11.1449, 28.8844, 26.2595, 28.2794, 28.8844, 27.555, 28.9455, 28.8845, 28.8845]
Nodal prices at time 2: [7.4682, 12.8732, 25.8844, 25.8844, 19.6842, 19.3428, 25.8844, 25.8845, 22.0733, 25.8844, 17.0733, 26.4348, 31.4348, 25.8845, 28.8134, 27.5396, 28.2176, 29.3449, 27.5396, 25.8845, 29.1701, 9.45774, 27.5396, 24.1932, 24.6193, 25.8845, 22.9241, 22.9024, 25.8845, 25.8845]
Nodal prices at time 3: [9.91918, 24.0282, 27.7912, 27.7912, 25.2772, 21.1805, 27.7912, 27.7912, 23.9564, 27.7913, 28.9564, 27.7913, 27.537, 27.7913, 27.7913, 27.7913, 27.7913, 27.7913, 27.7913, 27.7912, 28.5146, 15.2043, 27.9014, 27.4558, 27.3621, 27.7913, 25.3503, 24.8167, 27.7913, 27.7

In [17]:
actual_d = zeros(T,N)
for i in 1:T, j in 1:N
    actual_d[i,j] = value(d[i,j] - sum(del[j,:,i]) + sum(del[:,j,i]) + sum(gam[:,(i-1)*N+j]) - sum(gam[(i-1)*N+j,:]))
end
println(actual_d)

[0.0 43.3782 16.1176 17.1741 0.0 0.0 18.0776 3.44705 0.0 3.04866 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.105059 0.0 0.0 20.2169 32.2126 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 40.7436 17.136 18.7262 0.0 0.0 18.1534 4.26481 0.0 3.72276 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 23.3929 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 45.9256 16.7059 19.4788 0.0 0.0 12.9404 3.75258 0.0 3.9752 0.0 0.0 0.0 0.0 0.0 0.0 8.88178e-16 0.0 0.0 0.0 0.0 0.0 0.0 39.6662 0.0 0.0 0.0 0.0 0.0 0.0]


In [16]:
for i in 1:N, j in (i+1):N
    if Del[i,j] != 0
        println(value(del[i,j,1]) * value(del[j,i,1]))
        println(Del[i,j])
    end
end

0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0.0
10.0
0

In [44]:
value(sum(del))

349.3228550461449

In [5]:
# Optimization formulation

m_v = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(m_v, s[1:T,1:N] >= 0)                         # supply
@variable(m_v, d[1:T,1:N] >= 0)                         # demand
@variable(m_v, f[1:N,1:N,1:T] >= 0)                     # flow
@variable(m_v, del[1:N,1:N,1:T] >= 0)                   # spatial virtual shift
@variable(m_v, gam[1:N*T,1:N*T] >= 0)                   # temporal virtual shift
@variable(m_v, theta[1:T,1:N])

# Flow constraints and spatial virtual shift constraints
for t in 1:T
    for i in 1:N, j in 1:N
        @constraint(m_v, f[i,j,t] <= F[i,j])            # flow capacity constraints
        #@constraint(m_v, del[i,j,t] <= Del[i,j])        # spatial virtual shift capacity constraints
        @constraint(m_v, del[i,j,t] == 0)

        if i > j && B[i,j] > 0
            @constraint(m_v, f[i,j,t] - f[j,i,t] == B[i,j]*(theta[t,j] - theta[t,i]))
            @constraint(m_v, theta[t,j] - theta[t,i] <= theta_diff_lim)
            @constraint(m_v, theta[t,i] - theta[t,j] <= theta_diff_lim)
        end
    end
end

# Temporal virtual shift constraints
for i in 1:N*T, j in 1:N*T
    @constraint(m_v, gam[i,j] == 0)       # temporal shift is one-directional
end

# Supply and demand capacity constraints
for i in 1:T, j in 1:N
    @constraint(m_v, s[i,j] <= S_st[i,j])
    @constraint(m_v, d[i,j] <= D_st[i,j])
end

# Node balance constraint
bals = Dict()
for i in 1:T
    for j in 1:N
        bals[(i,j)] = @constraint(m_v, s[i,j] - d[i,j] - sum(f[j,:,i]) + sum(f[:,j,i]) + sum(del[j,:,i]) - sum(del[:,j,i]) - sum(gam[:,(i-1)*N+j]) + sum(gam[(i-1)*N+j,:]) == 0)
    end
end

# Objecitve: social welfare
@objective(m_v, Min, sum(α_s_st.*s - α_d_st.*d) + sum(α_f.*f) + sum(α_δ.*del) + sum(α_γ.*gam));

Academic license - for non-commercial use only


In [6]:
println("=======No virtual shift capacity======")
optimize!(m_v)
println("Social welfare: ", -objective_value(m_v))
prices_novl = zeros(T,N)
for i in 1:T, j in 1:N
    prices_novl[i,j] = dual(bals[(i,j)])
end

for i in 1:T
    println("Nodal prices at time $(i): ", prices_novl[i,:])
end

=======No virtual shift capacity======
Academic license - for non-commercial use only
Social welfare: 5209.202217079066
Nodal prices at time 1: [17.4754, 18.1533, 14.0189, 26.1455, 21.5692, 29.8391, 25.5193, 34.7248, 35.6346, 41.0513, 40.6346, 32.3847, 31.0985, 35.6604, 35.959, 41.8904, 45.0352, 41.8259, 47.3753, 45.1853, 50.5344, 11.1449, 26.5957, 25.791, 29.4239, 34.4239, 29.682, 32.6383, 33.2614, 36.7273]
Nodal prices at time 2: [7.4682, 12.8732, -18.2753, 30.8968, 20.0625, 32.4535, 26.5134, 28.0976, 36.6361, 41.2079, 41.6361, 35.3282, 36.6938, 38.1509, 40.0994, 41.7907, 48.151, 45.4615, 43.9261, 40.7466, 58.5178, 9.45774, 27.5396, 22.4823, 27.7569, 32.7569, 22.9241, 26.4015, 29.3446, 25.8787]
Nodal prices at time 3: [9.91918, 10.8218, -5.26515, 22.6623, 20.0133, 24.5911, 27.7912, 24.7226, 28.0724, 33.3435, 33.0724, 30.7193, 27.537, 33.641, 35.7284, 40.1186, 39.2103, 43.469, 40.0405, 35.8663, 47.6926, 15.2043, 27.9014, 27.4133, 27.346, 32.346, 25.3503, 24.8407, 28.9297, 32.3957]


In [7]:
# Compute the LMP variance and mean absolute deviation (MAD) at each time point

std_novl = std(prices_novl,dims=2)
std_vl = std(prices_vl,dims=2)
mad_novl = zeros(T)
mad_vl = zeros(T)
range_novl = zeros(T)
range_vl = zeros(T)

for i in 1:T
    mad_novl[i] = mad(prices_novl[i,:],center=mean(prices_novl[i,:]),normalize=false)
    mad_vl[i] = mad(prices_vl[i,:],center=mean(prices_vl[i,:]),normalize=false)
    range_novl[i] = maximum(prices_novl[i,:]) - minimum(prices_novl[i,:])
    range_vl[i] = maximum(prices_vl[i,:]) - minimum(prices_vl[i,:])
end

println("LMP standard deviation at each time point with no virtual links: ", std_novl)
println("LMP standard deviation at each time point with virtual links: ", std_vl)
println("LMP MAD at each time point with no virtual links: ", mad_novl)
println("LMP MAD at each time point with virtual links: ", mad_vl)
println("LMP range at each time point with no virtual links: ", range_novl)
println("LMP range at each time point with virtual links: ", range_vl)

LMP standard deviation at each time point with no virtual links: [9.79366; 14.6238; 10.7041]
LMP standard deviation at each time point with virtual links: [4.4348; 5.67428; 4.08942]
LMP MAD at each time point with no virtual links: [6.41187, 7.83426, 5.02478]
LMP MAD at each time point with virtual links: [1.4164, 2.24331, 1.72059]
LMP range at each time point with no virtual links: [39.3895, 76.7931, 52.9577]
LMP range at each time point with virtual links: [21.7415, 23.9667, 19.0371]


In [8]:
println(value.(theta)[1,:])

[-1.52328, -1.52328, -0.158296, -0.000586757, -0.659468, 0.0, -0.0869844, 0.0, 1.14706, 1.7479, 1.14706, 1.68065, 1.68065, 2.03855, 2.03855, 2.44958, 2.44958, 2.03855, 2.03855, 2.03855, 1.7479, 1.69759, -0.998892, 1.90371, 0.720595, 0.720595, 0.0, 0.0, 3.66072, 6.602]


In [13]:
for i in 1:N
    print("$(i): ")
    println(D_st[:,i])
end

1: [0.0, 0.0, 0.0]
2: [23.3782, 20.7436, 25.9256]
3: [3.00802, 0.0, 5.85362]
4: [5.19554, 7.61508, 12.9122]
5: [0.0, 0.0, 0.0]
6: [0.0, 0.0, 0.0]
7: [21.827, 22.8585, 27.9395]
8: [44.4138, 41.3557, 47.3557]
9: [0.0, 0.0, 0.0]
10: [4.40337, 0.0, 9.536]
11: [0.0, 0.0, 0.0]
12: [5.44296, 7.34214, 11.0886]
13: [0.0, 0.0, 0.0]
14: [5.81744, 10.4422, 7.15915]
15: [14.284, 12.6535, 0.324405]
16: [5.33306, 2.28527, 4.25272]
17: [8.91759, 8.49279, 24.2347]
18: [4.82532, 2.06886, 3.04866]
19: [17.2053, 0.0, 2.93412]
20: [5.25179, 0.0, 0.0]
21: [24.2327, 21.3645, 26.164]
22: [0.0, 0.0, 0.0]
23: [11.5308, 6.36142, 4.65271]
24: [12.2126, 3.39287, 19.6662]
25: [0.0, 0.0, 0.0]
26: [0.0, 12.5725, 7.3736]
27: [0.0, 0.0, 0.0]
28: [0.0, 0.0, 0.0]
29: [1.75068, 9.25218, 8.27381]
30: [13.6553, 4.65959, 8.89051]


In [33]:
for i in 1:N, j in (i+1):N
    if B[i,j] > 0
        println("($(i),$(j)) & $(round(B[i,j],digits = 4)) & $(round(minimum([theta_diff_lim * B[i,j], F[i,j]]),digits = 4)) \\\\")
    end
end

(1,2) & 15.0 & 5.6921 \\
(1,3) & 4.9223 & 17.6822 \\
(2,4) & 5.2308 & 16.225 \\
(2,5) & 4.7059 & 18.554 \\
(2,6) & 5.0 & 17.0763 \\
(3,4) & 23.5294 & 3.7108 \\
(4,6) & 23.5294 & 3.7108 \\
(4,12) & 3.8462 & 23.4 \\
(5,7) & 7.1006 & 11.7 \\
(6,7) & 10.9589 & 7.6896 \\
(6,8) & 23.5294 & 3.7108 \\
(6,9) & 4.7619 & 18.9 \\
(6,10) & 1.7857 & 17.8571 \\
(6,28) & 15.0 & 5.6921 \\
(8,28) & 4.5872 & 18.7926 \\
(9,10) & 9.0909 & 9.9 \\
(9,11) & 4.7619 & 18.9 \\
(10,17) & 10.9589 & 7.6896 \\
(10,20) & 4.023 & 20.5626 \\
(10,21) & 12.069 & 6.8542 \\
(10,22) & 5.4745 & 14.8977 \\
(12,13) & 7.1429 & 12.6 \\
(12,14) & 3.1707 & 25.7721 \\
(12,15) & 5.9633 & 13.2883 \\
(12,16) & 4.158 & 19.7385 \\
(14,15) & 2.2624 & 22.6244 \\
(15,18) & 3.6364 & 22.1371 \\
(15,23) & 4.0 & 20.1246 \\
(16,17) & 4.4706 & 18.554 \\
(18,19) & 6.3415 & 12.886 \\
(19,20) & 12.069 & 6.8542 \\
(21,22) & 40.0 & 2.0125 \\
(22,24) & 3.8462 & 19.47 \\
(23,24) & 3.0067 & 26.97 \\
(24,25) & 2.2759 & 22.7586 \\
(25,26) & 1.8366 & 18.36

In [38]:
Fst_tvl[2,32]

10.0

In [28]:
i = 1; j = 2
minimum([theta_diff_lim * B[i,j], F[i,j]])

5.692099788303082

In [43]:
α_γ[2,32]

3